In [156]:
import pandas as pd
import numpy as np
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
data_dir = 'data'
lb = 'iwgsc_4_2_19/'
output_fasta = data_dir + '/res/' + lb + '/sRNA_DEG.fasta'

In [157]:
results_file = data_dir + '/res/' + lb + '/Results.txt'
df_res = pd.read_csv(results_file,sep="\t")
#df_res = df_res[df_res.DicerCall.isin(['20','21','22'])]
print('Res len:',len(df_res.index))

('Res len:', 38496)


In [158]:
df_res.head()

,#Locus,Name,Length,Reads,RPM,UniqueReads,FracTop,Strand,MajorRNA,MajorRNAReads,...,DicerCall,MIRNA,PhaseScore,Short,Long,20,21,22,23,24
0,1A:209-460,Cluster_1,252,124,0.973,64,1.000,+,CGCGGAUUGCUCGAGCUGCUCACGCGGCGAGAGCGGGUCG,25,...,N,N2,-1.0,29,79,2,1,4,5,4
1,1A:688-753,Cluster_2,66,86,0.675,0,1.000,+,UAGACCGACUUUGUGAAAUG,5,...,N,N2,-1.0,8,56,7,1,3,6,5
2,1A:5808-6115,Cluster_3,308,239,1.876,103,1.000,+,GGUAUAACAUCAUAGGAU,31,...,N,N2,-1.0,116,59,13,14,18,9,10
3,1A:7054-7211,Cluster_4,158,216,1.695,69,1.000,+,CACAUGACUCUCGGCAACG,29,...,N,N2,-1.0,87,56,12,30,9,18,4
4,1A:7470-8084,Cluster_5,615,558,4.379,364,0.998,+,CCGGACCCUGUCGCCCCACGAGGCGCCA,51,...,N,N2,-1.0,56,416,16,21,12,25,12


In [159]:
df_res = df_res[['Name','MajorRNA','#Locus','MIRNA','DicerCall']]
df_res.head()

,Name,MajorRNA,#Locus,MIRNA,DicerCall
0,Cluster_1,CGCGGAUUGCUCGAGCUGCUCACGCGGCGAGAGCGGGUCG,1A:209-460,N2,N
1,Cluster_2,UAGACCGACUUUGUGAAAUG,1A:688-753,N2,N
2,Cluster_3,GGUAUAACAUCAUAGGAU,1A:5808-6115,N2,N
3,Cluster_4,CACAUGACUCUCGGCAACG,1A:7054-7211,N2,N
4,Cluster_5,CCGGACCCUGUCGCCCCACGAGGCGCCA,1A:7470-8084,N2,N


In [160]:
deg_file = data_dir + '/res/' + lb + 'diffexpr-results.0.01.csv'
df_deg = pd.read_csv(deg_file,sep=",",comment='#')
print('DEG len:',len(df_deg.index))

('DEG len:', 15187)


In [161]:
up = len(df_deg[(~df_deg.Gene.str.contains('Clust')) & (df_deg.log2FoldChange > 0)].index)
down = len(df_deg[(~df_deg.Gene.str.contains('Clust')) & (df_deg.log2FoldChange < 0)].index)
(up, down)

(13719, 752)

In [162]:
up = len(df_deg[(df_deg.Gene.str.contains('Clust')) & (df_deg.log2FoldChange > 0)].index)
down = len(df_deg[(df_deg.Gene.str.contains('Clust')) & (df_deg.log2FoldChange < 0)].index)
(up, down)

(490, 226)

In [163]:
df_deg['seq'] = ''
for k,v in df_deg.iterrows():
    if 'Cluster' in v.Gene:
        df_deg.loc[k, 'Gene'] = v.Gene.split('_')[0] + '_' + v.Gene.split('_')[1]
    else:
        df_deg.loc[k, 'Gene']  = v.Gene.split('_')[0] 
        df_deg.loc[k, 'seq'] = v.Gene.split('_')[0] 

In [164]:

df_deg.tail()

,Gene,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,X21dpiR1C.trimmed,X21dpiR3C.trimmed,X21dpiR1T.trimmed,X21dpiR3T.trimmed,seq
15182,CUUAAGGUAGCCAAAUGCCU,22.179823,2.671161,0.827816,3.226759,0.001252,0.009973,2.165950,9.954040,46.184556,30.414747,CUUAAGGUAGCCAAAUGCCU
15183,Cluster_2593,142.617625,-1.195305,0.370452,-3.226612,0.001253,0.009977,199.267398,197.658788,67.553529,105.990786,
15184,CGAAAGAUGGUGAACU,220.696039,0.771691,0.239192,3.226241,0.001254,0.009990,158.114349,167.796669,261.942257,294.930882,CGAAAGAUGGUGAACU
15185,UCGGUCCAUAUCCGGUGUGGGC,15.061301,3.032649,0.940069,3.225987,0.001255,0.009997,2.165950,4.266017,37.223373,16.589862,UCGGUCCAUAUCCGGUGUGGGC
15186,UAAAUACAGGCGAGAGACCGAUAGCGAACAAGUA,42.743617,2.260646,0.700760,3.225993,0.001255,0.009997,10.829750,18.486074,99.262329,42.396314,UAAAUACAGGCGAGAGACCGAUAGCGAACAAGUA


In [165]:
df_deg_res = pd.merge(df_deg, df_res,left_on='Gene', right_on='Name' , how='left')
#only some cols
#df_deg_res = df_deg_res['#Locus','Name','MajorRNA','log2FoldChange']


In [166]:
df_deg_res.tail()

,Gene,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,X21dpiR1C.trimmed,X21dpiR3C.trimmed,X21dpiR1T.trimmed,X21dpiR3T.trimmed,seq,Name,MajorRNA,#Locus,MIRNA,DicerCall
15182,CUUAAGGUAGCCAAAUGCCU,22.179823,2.671161,0.827816,3.226759,0.001252,0.009973,2.165950,9.954040,46.184556,30.414747,CUUAAGGUAGCCAAAUGCCU,NaN,NaN,NaN,NaN,NaN
15183,Cluster_2593,142.617625,-1.195305,0.370452,-3.226612,0.001253,0.009977,199.267398,197.658788,67.553529,105.990786,,Cluster_2593,UUUCGUGAAGGCUUCUAACUU,1B:578093667-578094261,N5,21
15184,CGAAAGAUGGUGAACU,220.696039,0.771691,0.239192,3.226241,0.001254,0.009990,158.114349,167.796669,261.942257,294.930882,CGAAAGAUGGUGAACU,NaN,NaN,NaN,NaN,NaN
15185,UCGGUCCAUAUCCGGUGUGGGC,15.061301,3.032649,0.940069,3.225987,0.001255,0.009997,2.165950,4.266017,37.223373,16.589862,UCGGUCCAUAUCCGGUGUGGGC,NaN,NaN,NaN,NaN,NaN
15186,UAAAUACAGGCGAGAGACCGAUAGCGAACAAGUA,42.743617,2.260646,0.700760,3.225993,0.001255,0.009997,10.829750,18.486074,99.262329,42.396314,UAAAUACAGGCGAGAGACCGAUAGCGAACAAGUA,NaN,NaN,NaN,NaN,NaN


In [167]:

df_deg_res = df_deg_res.replace(np.nan, '', regex=True)
df_deg_res = df_deg_res.replace('nan', '', regex=True)
df_deg_res.seq = df_deg_res.seq.astype(str) + df_deg_res.MajorRNA.astype(str)

In [168]:
df_deg_res = df_deg_res.drop(['Name'], axis=1).drop(['MajorRNA'], axis=1)

In [169]:
df_deg_res.tail()

,Gene,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,X21dpiR1C.trimmed,X21dpiR3C.trimmed,X21dpiR1T.trimmed,X21dpiR3T.trimmed,seq,#Locus,MIRNA,DicerCall
15182,CUUAAGGUAGCCAAAUGCCU,22.179823,2.671161,0.827816,3.226759,0.001252,0.009973,2.165950,9.954040,46.184556,30.414747,CUUAAGGUAGCCAAAUGCCU,,,
15183,Cluster_2593,142.617625,-1.195305,0.370452,-3.226612,0.001253,0.009977,199.267398,197.658788,67.553529,105.990786,UUUCGUGAAGGCUUCUAACUU,1B:578093667-578094261,N5,21
15184,CGAAAGAUGGUGAACU,220.696039,0.771691,0.239192,3.226241,0.001254,0.009990,158.114349,167.796669,261.942257,294.930882,CGAAAGAUGGUGAACU,,,
15185,UCGGUCCAUAUCCGGUGUGGGC,15.061301,3.032649,0.940069,3.225987,0.001255,0.009997,2.165950,4.266017,37.223373,16.589862,UCGGUCCAUAUCCGGUGUGGGC,,,
15186,UAAAUACAGGCGAGAGACCGAUAGCGAACAAGUA,42.743617,2.260646,0.700760,3.225993,0.001255,0.009997,10.829750,18.486074,99.262329,42.396314,UAAAUACAGGCGAGAGACCGAUAGCGAACAAGUA,,,


In [176]:
df_deg_res[df_deg_res.MIRNA == 'Y']

,Gene,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,X21dpiR1C.trimmed,X21dpiR3C.trimmed,X21dpiR1T.trimmed,X21dpiR3T.trimmed,seq,#Locus,MIRNA,DicerCall
607,Cluster_33754,2990.897346,-1.092349,0.160402,-6.810064,9.755511e-12,1.940512e-09,4201.942964,3943.221716,1903.217292,1915.207413,UGUAGAUACUCCCUAAGGCUU,7B:9130339-9130430,Y,21
3093,Cluster_9839,323.635553,-1.796546,0.330720,-5.432230,5.565415e-08,2.175441e-06,444.019746,561.692239,122.009946,166.820280,AGGCGGUUACCUUGGCUAGC,2D:360310978-360311149,Y,20
3760,Cluster_32170,289.998277,-1.198635,0.228737,-5.240240,1.603676e-07,5.156837e-06,429.941071,378.253508,181.291615,170.506916,UGUAGAUACUCCCUAAGGCUU,7A:72699174-72699257,Y,21
5506,Cluster_17194,215.195641,-1.343073,0.275625,-4.872829,1.100114e-06,2.415538e-05,309.730847,308.575230,136.485702,105.990786,UAGCCAAGAAUGACUUGCCU,4A:494638245-494638452,Y,21
9590,Cluster_18555,867.793001,-1.400455,0.326071,-4.294941,1.747400e-05,2.203426e-04,1188.023565,1330.997305,385.330846,566.820288,GGUGGGUCUUCUUGGCUAGC,4B:117381111-117381289,Y,20
10489,Cluster_35737,222.177723,-1.029754,0.250739,-4.106881,4.010378e-05,4.623595e-04,324.892497,271.603082,147.514850,144.700464,UGUAGAUACUCCCUAAGGCUU,7D:68403780-68403911,Y,21
11310,Cluster_16054,12198.538549,-0.715709,0.182832,-3.914578,9.056256e-05,9.683117e-04,15209.300769,15123.030274,8648.919726,9812.903429,UACUGUGGGCACUUAUUUGACA,3D:544447377-544447613,Y,22
12061,Cluster_6291,311.504475,-1.578261,0.421775,-3.741952,1.825961e-04,1.830805e-03,598.885170,335.593338,158.543998,152.995395,UGCCAAAGGAGAGUUGCCCUG,2A:652328872-652329011,Y,21
12236,Cluster_28963,839.633466,-0.659671,0.178066,-3.704650,2.116828e-04,2.092091e-03,1013.664591,1043.752160,673.467329,627.649783,UCGCUUGGUGCAGAUCGGGAC,6B:103294141-103294344,Y,21
12967,Cluster_16796,3455.085821,-0.942752,0.263969,-3.571450,3.550097e-04,3.310833e-03,5185.284255,3909.093580,2338.868624,2387.096824,AUGCCGUGUUGUUCUGAAAGAA,4A:154525375-154525580,Y,22


In [175]:
len(df_deg_res[df_deg_res.MIRNA == 'Y'].index)

11

In [171]:
df_deg_res.to_csv(data_dir + '/res/' + lb + '/deg_data.csv', sep='\t', index=None)

In [172]:
len(df_deg_res.index)

15187

In [179]:
#save fasta
buffer_seqs = []
for k,v in df_deg_res.iterrows():
    new_id = v.Gene
    new_desc = 'chr:%s log2fc:%f size:%i dicercall:%s miRNA:%s' % (v['#Locus'],  v.log2FoldChange, len(v.seq), v.DicerCall, v.MIRNA)
    seq = v.seq
    record = SeqRecord(Seq(seq), id=new_id, description=new_desc)
    buffer_seqs.append(record)
SeqIO.write(buffer_seqs, output_fasta, "fasta")

15187

In [180]:
output_fasta

'data/res/iwgsc_4_2_19//sRNA_DEG.fasta'

In [41]:
df_res = pd.merge(df_deg, df_blast[['qseqid','sseqid']], left_on='Gene', right_on='qseqid', how='left')
df_res = df_res.drop(['qseqid'], axis=1)
df_res.rename(columns={'sseqid': 'miRBASE'}, inplace=True)
df_res = df_res.replace(np.nan, '', regex=True)
print(len(df_res.index))
df_res.head()

NameError: name 'df_blast' is not defined

In [6]:
#count DEG len for downregulated
df_down = df_deg[df_deg.log2FoldChange < 0]
y21 = len(df_down[df_down.Gene.isin(df_res[df_res.DicerCall.isin(['21'])].Name)].index)
y22 = len(df_down[df_down.Gene.isin(df_res[df_res.DicerCall.isin(['22'])].Name)].index)
y23 = len(df_down[df_down.Gene.isin(df_res[df_res.DicerCall.isin(['23'])].Name)].index)
y24 = len(df_down[df_down.Gene.isin(df_res[df_res.DicerCall.isin(['24'])].Name)].index)
(y21, y22, y23, y24)

(159, 37, 5, 1974)

In [20]:
#count DEG len for upregulated
df_up = df_deg[df_deg.log2FoldChange > 0]
y21 = len(df_up[df_up.Gene.isin(df_res[df_res.DicerCall.isin(['21'])].Name)].index)
y22 = len(df_up[df_up.Gene.isin(df_res[df_res.DicerCall.isin(['22'])].Name)].index)
y23 = len(df_up[df_up.Gene.isin(df_res[df_res.DicerCall.isin(['23'])].Name)].index)
y24 = len(df_up[df_up.Gene.isin(df_res[df_res.DicerCall.isin(['24'])].Name)].index)
(y21, y22, y23, y24)

(39, 4, 1, 13)

In [4]:
#count DEG len
y21 = len(df_deg[df_deg.Gene.isin(df_res[df_res.DicerCall.isin(['21'])].Name)].index)
y22 = len(df_deg[df_deg.Gene.isin(df_res[df_res.DicerCall.isin(['22'])].Name)].index)
y23 = len(df_deg[df_deg.Gene.isin(df_res[df_res.DicerCall.isin(['23'])].Name)].index)
y24 = len(df_deg[df_deg.Gene.isin(df_res[df_res.DicerCall.isin(['24'])].Name)].index)
(y21, y22, y23, y24)

(263, 45, 9, 2320)